In [34]:
#!/usr/bin/env python
"""
@file    simulation.py
@author  Shubajit Saha
Date: 20-11-2016
"""


from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import subprocess
import random
from Queue import PriorityQueue
from xml.dom import minidom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# we need to import python modules from the $SUMO_HOME/tools directory
try:
    ## Set __file__ as current file name
    __file__ = "/home/shuvojit/Project/Simulation/DynamicRouteSuggestionSystem/docs/tutorial/traci_tls/Simulation-Notebook.ipnb"
    sys.path.append(os.path.join(os.path.dirname(
        __file__), '..', '..', '..', '..', "tools"))  # tutorial in tests
    sys.path.append(os.path.join(os.environ.get("SUMO_HOME", os.path.join(
        os.path.dirname(__file__), "..", "..", "..")), "tools"))  # tutorial in docs
    from sumolib import checkBinary
except ImportError:
    sys.exit(
        "please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")

import traci

In [37]:
class Parameters:
    inf = 1e8
    delta = 1.5 #mindist multiplier
    beta = 0.5 #min travel tims vs min path trade off
    alpha = 0.01 #learning rate
    gamma = 0.5 #discount factor
    
    epsilon =  0.5 #exploartion vs exploitation
    cona = 10 #constant multiplier for congestion
    conb = 4 #length increases quadratically to the congestion
    iterations = 1000

In [10]:
#Generates Route file for example graph

def generate_routefile():
    random.seed(42)  # make tests reproducible
    N = Parameters.iterations  # number of time steps
    # demand per second from different directions
    
    with open("testdata/sim.rou.xml", "w") as routes:
        print("""<routes>
        <vType id="vtype0" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" guiShape="passenger"/>
        <vType id="vtype1" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" guiShape="passenger"/>
        <route id="r1to2" edges="1to2 " />
        <route id="r1to3" edges="1to3 " />
        <route id="r1to4" edges="1to4 " />
        <route id="r1to5" edges="1to4 4to5 " />
        <route id="r1to6" edges="1to4 4to6 " />
        <route id="r1to7" edges="1to4 4to7 " />
        <route id="r1to8" edges="1to4 4to7 7to8 " />
        <route id="r1to9" edges="1to4 4to7 7to9 " />
        <route id="r1to10" edges="1to4 4to7 7to10 " />
        <route id="r2to3" edges="2to3 " />
        <route id="r2to4" edges="2to4 " />
        <route id="r2to5" edges="2to5 " />
        <route id="r2to6" edges="2to5 5to6 " />
        <route id="r2to7" edges="2to5 5to7 " />
        <route id="r2to8" edges="2to5 5to8 " />
        <route id="r2to9" edges="2to5 5to8 8to9 " />
        <route id="r2to10" edges="2to5 5to8 8to10 " />
        <route id="r3to4" edges="3to4 " />
        <route id="r3to5" edges="3to5 " />
        <route id="r3to6" edges="3to6 " />
        <route id="r3to7" edges="3to6 6to7 " />
        <route id="r3to8" edges="3to6 6to8 " />
        <route id="r3to9" edges="3to6 6to9 " />
        <route id="r3to10" edges="3to6 6to9 9to10 " />
        <route id="r4to5" edges="4to5 " />
        <route id="r4to6" edges="4to6 " />
        <route id="r4to7" edges="4to7 " />
        <route id="r4to8" edges="4to7 7to8 " />
        <route id="r4to9" edges="4to7 7to9 " />
        <route id="r4to10" edges="4to7 7to10 " />
        <route id="r5to6" edges="5to6 " />
        <route id="r5to7" edges="5to7 " />
        <route id="r5to8" edges="5to8 " />
        <route id="r5to9" edges="5to8 8to9 " />
        <route id="r5to10" edges="5to8 8to10 " />
        <route id="r6to7" edges="6to7 " />
        <route id="r6to8" edges="6to8 " />
        <route id="r6to9" edges="6to9 " />
        <route id="r6to10" edges="6to9 9to10 " />
        <route id="r7to8" edges="7to8 " />
        <route id="r7to9" edges="7to9 " />
        <route id="r7to10" edges="7to10 " />
        <route id="r8to9" edges="8to9 " />
        <route id="r8to10" edges="8to10 " />
        <route id="r9to10" edges="9to10 " />""", file=routes)
        lastVeh = 0
        vehNr = 0
        count = [[0 for j in range(3)] for i in range(3)]
        for i in range(N):
            src = random.randint(1,3)
            dest = random.randint(8,10)
            rid = "r"+str(src)+"to"+str(dest)
            print('    <vehicle id="vid#%i#%i#%i" type="vtype1" route="%s" depart="%i" />' % (
                vehNr,src,dest,rid, i), file=routes)
            vehNr += 1
            lastVeh = i
            count[src-1][dest-8] = count[src-1][dest-8]+1; #input specific

        #print("Vehicles Generated: ",pd.DataFrame(count))
        print("</routes>", file=routes)

In [43]:
def preProcess(g,w):
    N = len(g)
    dist = [[Parameters.inf for j in range(N)]for i in range(N)]
    for i in range(N): #for all vertices
        dist[i][i] = 0
        pq = PriorityQueue()
        pq.put((0,i))
        while pq.qsize() > 0:
            u = pq.get()[1]
            #print "u: ",u
            for j in range(len(g[u])):
                v = g[u][j]
                d = w[u][j]
                if dist[i][v] > dist[i][u] + d:
                    dist[i][v] = dist[i][u] + d
                    pq.put((dist[i][v],v))
    
    #Find Initial Qvalues
    qf = [[[dist[i][j] for k in range(N)]for j in range(N)] for i in range(N)]
    prev = [[[0 for k in range(N)]for j in range(N)] for i in range(N)]

#     alpha = 0.01
#     gamma = 0.5
    alpha = Parameters.alpha
    gamma = Parameters.gamma
    for iteration in range(1000):
        for src in range(N):
            for des in range(N):
                if src == des:
                    continue
                else:
                    for action in range(len(g[src])):
                        s_dash = g[src][action]
                        reward = -(w[src][action] + dist[g[src][action]][des])
                        maxim = -1e100
                        if s_dash == des:
                            maxim = 100 - qf[src][des][s_dash]
                        else:
                            for a_dash in range(len(g[s_dash])):
                                maxim = max(maxim,qf[s_dash][des][g[s_dash][a_dash]] -  qf[src][des][g[src][action]])
                        qf[src][des][g[src][action]] =  qf[src][des][g[src][action]] + alpha*(reward + gamma*maxim)


        maxdiff = -1e8 
        for i in range(N):
            for j in range(N):
                for k in range(len(g[i])):
                    diff = abs(prev[i][j][g[i][k]]-qf[i][j][g[i][k]])
                    maxdiff = max(maxdiff,diff)
                    prev[i][j][g[i][k]] = qf[i][j][g[i][k]]
        if maxdiff < .0001:
            print ("coverged with in expected deviation in #iterations: ",iteration+1)
            break
    return dist,qf

In [ ]:
def findAction(src,dest,w,dist,qf,g):
    action = -1
    for i in range(len(q[src][dest])):
        if dist[g[src][i]][dest] != Parameters.inf and qf[src][des][i] > maxim:
            maxim = qf[src][des][i]
            action = i
    return action

In [ ]:
def processVehicle(vid,route,curedge,edgeweights,sdist,qvalues,g):
    '''process the vid return next edge to be taken and reroute
        by updating the qvalues
    '''

    L = len(route)
    src = int(curedge.split("to")[1])
    dest = int(route[L-1].split("to")[1])
    src = src - 1
    dest = dest - 1

    if src == dest: #reached destination
        #assert route[0] == curedge, "single route element not equal to cur edge"
        '''
            store the travel time etc in a queue
        '''
        return None
 

    action = findAction(src,dest,edgeweights,sdist,qvalues)
    print("(src = %i,destination = %i ,action = %d): "%(src,dest,action))

    

    #update qvalues for all Q(s,a) pairs
    #Q(s,a) = Q(s,a) + alpha(r+ gamma(Max(Q(s',a')-Q(s,a))))

    naction = findAction(action,dest,edgeweights,sdist,qvalues)
#     print("next action is: ",naction)
#     N = traci.junction.getIDCount()
#     for i in range(N):
#         if edgeweights[src][i] > 0 and sdist[i][dest] < Parameters.inf:
#             # assert 1 == 2,"ki hochche"
#             # print("jai ma")
#             qvalues[src][dest][i] = qvalues[src][dest][i] + Parameters.alpha*(calReward(src,dest,action,sdist,qvalues)
#                                     + Parameters.gamma*qvalues[action][dest][naction] - qvalues[src][dest][i])

    return naction


In [ ]:
#Route Choice Server
def run(edgeweights,sdist,qvalues,g):
    """execute the TraCI control loop"""
    step = 0
    edges = traci.edge.getIDList();
    print("num edgees = %i",traci.edge.getIDCount())
    print("edge list",edges)

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep() 
        print("running step no. %i",step)

        numedges = 24
        for i in range(numedges):
            loopid = "loop"+str(i)
            vids = traci.inductionloop.getLastStepVehicleIDs(loopid)
            src = (edges[i].split('to')[1])
            curedge = edges[i]
            if vids :
                print(''.join(vids))
                for vid in vids:
                    route = traci.vehicle.getRoute(vid)
                    print(vid+" old route: "+' '.join(route))
                    dest = int(route[len(route)-1].split('to')[1])
                    
                    #Get New Route Suggestion
                    nexthop = processVehicle(vid,route,edges[i],edgeweights,sdist,qvalues,g)

                    if nexthop  and nexthop != dest: #currently in last edge so do nothing
                        print("nexthop: ",nexthop," dest: ",dest,nexthop == dest)
                        nextedge = str(src)+"to"+str(nexthop)
                        rid = "r"+str(nexthop)+"to"+str(dest)
                        print("new route id: ",rid)
                        routeedges = traci.route.getEdges(rid)
                        routeedges.insert(0,nextedge)
                        routeedges.insert(0,curedge)
                        print("new route edges: ",routeedges)
                        traci.vehicle.setRoute(vid,routeedges)
                        newroute = traci.vehicle.getRoute(vid)
                        print(vid+" new route: "+' '.join(newroute))
                    
            # if step%10 == 0:
            #     printQValues(qvalues)
        step += 1
    return

In [47]:
if __name__ == "__main__":
        sumoBinary = checkBinary('sumo-gui')
        #genrate route file
        generate_routefile()
        
        #Start Simulation
        traci.start([sumoBinary, "-c", "testdata/sim.sumocfg",
                             "--tripinfo-output", "simtripinfo.xml"])
        
        
        g=[[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7],[6,7,8],[7,8,9],[8,9],[9],[]]
        #change weight test
        w = [[10, 10, 1], [10, 10, 10], [10, 10, 10], [10, 10, 10], [10, 10, 10], [10, 10, 10], [10, 10, 10], [1, 10], [1], []]      
        N = len(g)
        dist,qf = preProcess(g,w)
#         display(pd.DataFrame(dist))
#         display(pd.DataFrame(qf))
        
        run(w,dist,qf,g)
        print("simulation completed successfully!!")
        
        traci.close()
        sys.stdout.flush()


Could not connect to TraCI server at localhost:57699 [Errno 111] Connection refused
 Retrying in 1 seconds


NameError: name 'run' is not defined

In [50]:
file = open("log.txt",'w')
sys.stdout = file
print("saha")
file.close()